# Combined buoy and Hindcast Dataframe
Create a dataframe by combining cleaned historical buoy data, with features from Hindcast data that was given as grid point close to location of buoy data. Features to incorporate are:
- ETOT; Total variance of total spectrum (M^2) 
- ETTSea
- ETTSw
- VMD; Vector Mean Direction: direction to wich waves are traveling, clockwise from north in degrees
- VMDSw; 
- VMDSea

In wave spectral analysis locally generated wind waves are refered to as seas and waves travelling from distant sources are refered to as swell. Generally seas contain shorter periods and steeper waves and swells contain longer period and less steep waves. https://natural-resources.canada.ca/sites/www.nrcan.gc.ca/files/canmetenergy/files/pubs/CHC-TR-051.pdf
From the Hindcast grid point , ETOT refers to the total energy in the total spectrum (sea and swell combined) Then there is the separate Energy from the Sea and Swell . 

All features will be included to start, then columns that are not relevent may be dropped. 

The data will be combined on the datetime so that they align. Hindcast data is give for three hour intervals, as such missing values will be imputed using forward fill, so hour intervals will contain the same value for the three hour interval. 
Possibly look into imputing NaN values using averages instead. 

Possible inclusion of tidal data: Consideration, tidal data measured from land station and not same location. 

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#import gridpoint data from gridpoint closest to buoy. 
file_path = 'GN018918.opr_dump'
with open(file_path, 'r') as file:
    data1 = file.read()

In [3]:
column_names = ["CCYYMM", "DDHHmm", "WD", "WS", "ETOT", "TP", "VMD", "ETTSea", "TPSea", "VMDSea", 
                "ETTSw", "TPSw", "VMDSw", "MO1", "MO2", "HS", "DMDIR", "ANGSPR", "INLINE"]

# Read data into a DataFrame
df_hind = pd.read_csv(file_path, skiprows=25, delim_whitespace=True, names=column_names)

In [4]:
df_hind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105190 entries, 0 to 105189
Data columns (total 19 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   CCYYMM  105190 non-null  int64  
 1   DDHHmm  105190 non-null  int64  
 2   WD      105190 non-null  float64
 3   WS      105190 non-null  float64
 4   ETOT    105190 non-null  float64
 5   TP      105190 non-null  float64
 6   VMD     105190 non-null  float64
 7   ETTSea  105190 non-null  float64
 8   TPSea   105190 non-null  float64
 9   VMDSea  105190 non-null  float64
 10  ETTSw   105190 non-null  float64
 11  TPSw    105190 non-null  float64
 12  VMDSw   105190 non-null  float64
 13  MO1     105190 non-null  float64
 14  MO2     105190 non-null  float64
 15  HS      105190 non-null  float64
 16  DMDIR   105190 non-null  float64
 17  ANGSPR  105190 non-null  float64
 18  INLINE  105190 non-null  float64
dtypes: float64(17), int64(2)
memory usage: 15.2 MB


In [5]:
df_hind[['CCYYMM','DDHHmm']]

,CCYYMM,DDHHmm
0,198001,10600
1,198001,10900
2,198001,11200
3,198001,11500
4,198001,11800
...,...,...
105185,201512,310900
105186,201512,311200
105187,201512,311500
105188,201512,311800


In [6]:
#use zfill() as it adds zeros to the begining of the string, to add 0 to days 1-9 for each month. 
#convert DDHHmm to string first to use zfill()
df_hind['DDHHmm'] = df_hind['DDHHmm'].astype(str).str.zfill(6) #ensures 6 characters and will add 0 to beginning if not

In [7]:
df_hind['DDHHmm']

0         010600
1         010900
2         011200
3         011500
4         011800
           ...  
105185    310900
105186    311200
105187    311500
105188    311800
105189    312100
Name: DDHHmm, Length: 105190, dtype: object

In [8]:
def process_datetime_columns(df, CCYYMM_col, DDHHmm_col):
    # Convert 'CCYYMM' and 'DDHHmm' to datetime format and combine into one column
    df['year_month'] = pd.to_datetime(df[CCYYMM_col].astype(str), format='%Y%m')
    
    # Extract components of day, hour, and minute, convert to string to slice, then convert back to int
    df['DDHHmm_str'] = df[DDHHmm_col].astype(str)
    df['day'] = df['DDHHmm_str'].str[:2].astype(int)
    df['hour'] = df['DDHHmm_str'].str[2:4].astype(int)
    df['minute'] = df['DDHHmm_str'].str[4:].astype(int)

    # Convert each component into timedelta to be combined with date time
    day_timedelta = pd.to_timedelta(df['day'], unit='D')
    hour_timedelta = pd.to_timedelta(df['hour'], unit='H')
    minute_timedelta = pd.to_timedelta(df['minute'], unit='m')

    # Combine into one column
    df['Datetime'] = pd.to_datetime(df['year_month']) + day_timedelta + hour_timedelta + minute_timedelta

    # Drop specified columns
    df = df.drop(['year_month', 'CCYYMM', 'DDHHmm', 'DDHHmm_str', 'day', 'hour', 'minute'], axis=1)

    return df


In [9]:
df_hind = process_datetime_columns(df_hind, 'CCYYMM', 'DDHHmm')

In [10]:
df_hind['Datetime']

0        1980-01-02 06:00:00
1        1980-01-02 09:00:00
2        1980-01-02 12:00:00
3        1980-01-02 15:00:00
4        1980-01-02 18:00:00
                 ...        
105185   2016-01-01 09:00:00
105186   2016-01-01 12:00:00
105187   2016-01-01 15:00:00
105188   2016-01-01 18:00:00
105189   2016-01-01 21:00:00
Name: Datetime, Length: 105190, dtype: datetime64[ns]

In [11]:
#import historical buoy data
df_buoy = pd.read_csv('../buoy_data/laperouse_stn46206_clean.csv')

In [12]:
df_buoy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237882 entries, 0 to 237881
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   STN_ID     237882 non-null  object 
 1   DATE       237882 non-null  object 
 2   Q_FLAG     237882 non-null  int64  
 3   LATITUDE   237882 non-null  float64
 4   LONGITUDE  237882 non-null  float64
 5   DEPTH      237882 non-null  float64
 6   VWH$       237882 non-null  float64
 7   VCMX       237882 non-null  float64
 8   VTP$       237882 non-null  float64
 9   WDIR       237882 non-null  float64
 10  WSPD       237882 non-null  float64
 11  GSPD       237882 non-null  float64
 12  WDIR.1     237882 non-null  float64
 13  WSPD.1     237882 non-null  float64
 14  GSPD.1     237882 non-null  float64
 15  ATMS       237882 non-null  float64
 16  DRYT       237882 non-null  float64
 17  SSTP       237882 non-null  float64
 18  YEAR       237882 non-null  int64  
dtypes: float64(15), int64(2

In [13]:
#convert Date to datetime and rename "Datetime"
df_buoy['DATE']=pd.to_datetime(df_buoy['DATE'])
df_buoy=df_buoy.rename(columns ={'DATE':'Datetime'})

In [14]:
df_buoy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237882 entries, 0 to 237881
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   STN_ID     237882 non-null  object        
 1   Datetime   237882 non-null  datetime64[ns]
 2   Q_FLAG     237882 non-null  int64         
 3   LATITUDE   237882 non-null  float64       
 4   LONGITUDE  237882 non-null  float64       
 5   DEPTH      237882 non-null  float64       
 6   VWH$       237882 non-null  float64       
 7   VCMX       237882 non-null  float64       
 8   VTP$       237882 non-null  float64       
 9   WDIR       237882 non-null  float64       
 10  WSPD       237882 non-null  float64       
 11  GSPD       237882 non-null  float64       
 12  WDIR.1     237882 non-null  float64       
 13  WSPD.1     237882 non-null  float64       
 14  GSPD.1     237882 non-null  float64       
 15  ATMS       237882 non-null  float64       
 16  DRYT       237882 no

# Join Hindcast Data and Buoy Data

In [15]:
#check for duplicates in datetime columns 
duplicates_buoy_date =df_buoy['Datetime'].duplicated().sum()
duplicates_hind_date =df_hind['Datetime'].duplicated().sum()
print('duplicates date buoy data:', duplicates_buoy_date)
print('duplicates date hindcast data:',duplicates_hind_date)

duplicates date buoy data: 0
duplicates date hindcast data: 0


In [16]:
#create column to join df_buoy and df_hind, excluding minutes
df_buoy['DATE'] = df_buoy['Datetime'].dt.strftime('%Y-%m-%d %H')
df_hind['DATE'] = df_hind['Datetime'].dt.strftime('%Y-%m-%d %H') 


In [17]:
print(df_buoy['DATE'].describe())
print(df_hind['DATE'].describe())
#df_buoy will have less unique values because minutes are now taken out of record and it is just to the hour.

count            237882
unique           237347
top       2009-04-18 04
freq                  2
Name: DATE, dtype: object
count            105190
unique           105190
top       1980-01-02 06
freq                  1
Name: DATE, dtype: object


In [18]:
#join df_hind to df_buoy using left join: all rows in df_buoy will be in resulting data frame,
#with corresponding rows from df_hind
main_df = pd.merge(df_buoy, df_hind, how = 'left', on = 'DATE')

In [19]:
main_df.head(20)

,STN_ID,Datetime_x,Q_FLAG,LATITUDE,LONGITUDE,DEPTH,VWH$,VCMX,VTP$,WDIR,...,ETTSw,TPSw,VMDSw,MO1,MO2,HS,DMDIR,ANGSPR,INLINE,Datetime_y
0,C46206,1988-11-22 22:34:00,1,48.83,126.0,73.0,5.42,8.4,11.64,234.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,C46206,1988-11-23 00:33:00,1,48.83,126.0,73.0,5.89,9.0,14.22,263.0,...,0.844,12.229,63.7,0.682,0.564,3.981,67.0,0.8075,0.7153,1988-11-23 00:00:00
2,C46206,1988-11-23 01:34:00,1,48.83,126.0,73.0,6.49,10.5,12.80,277.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,C46206,1988-11-23 02:34:00,1,48.83,126.0,73.0,7.34,12.1,15.06,275.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,C46206,1988-11-23 04:34:00,1,48.83,126.0,73.0,7.71,13.0,14.22,295.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,C46206,1988-11-23 05:34:00,1,48.83,126.0,73.0,8.05,13.9,14.22,288.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
6,C46206,1988-11-23 06:34:00,1,48.83,126.0,73.0,8.51,17.5,16.00,281.0,...,0.707,13.729,83.1,0.867,0.793,4.366,70.9,0.7644,0.6795,1988-11-23 06:00:00
7,C46206,1988-11-23 07:34:00,1,48.83,126.0,73.0,8.70,13.9,15.06,297.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8,C46206,1988-11-23 08:34:00,1,48.83,126.0,73.0,7.23,13.3,16.00,283.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
9,C46206,1988-11-23 09:34:00,1,48.83,126.0,73.0,7.32,12.2,16.00,289.0,...,0.800,14.450,82.6,1.161,1.093,4.952,67.1,0.7056,0.6109,1988-11-23 09:00:00


## Drop rows outside of overlapping time range and impute NaN values
Appears that data frames have merged successfully with NaN values approximately inbetween every third row as expected with frequency of DATE in df_hind being 3 hour intervals. 

Next NaN values will be imputed using forward fill method so values given for each three hour interval will be the same for each hour in that interval. 

In [20]:
#max date from df_hind
main_df['Datetime_y'].max()

Timestamp('2016-01-01 21:00:00')

In [21]:
max_datetime_x =main_df['Datetime_y'].max()
main_df = main_df[main_df['Datetime_x'] <= max_datetime_x].copy()



In [22]:
main_df.head(20)

,STN_ID,Datetime_x,Q_FLAG,LATITUDE,LONGITUDE,DEPTH,VWH$,VCMX,VTP$,WDIR,...,ETTSw,TPSw,VMDSw,MO1,MO2,HS,DMDIR,ANGSPR,INLINE,Datetime_y
0,C46206,1988-11-22 22:34:00,1,48.83,126.0,73.0,5.42,8.4,11.64,234.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,C46206,1988-11-23 00:33:00,1,48.83,126.0,73.0,5.89,9.0,14.22,263.0,...,0.844,12.229,63.7,0.682,0.564,3.981,67.0,0.8075,0.7153,1988-11-23 00:00:00
2,C46206,1988-11-23 01:34:00,1,48.83,126.0,73.0,6.49,10.5,12.80,277.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,C46206,1988-11-23 02:34:00,1,48.83,126.0,73.0,7.34,12.1,15.06,275.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,C46206,1988-11-23 04:34:00,1,48.83,126.0,73.0,7.71,13.0,14.22,295.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,C46206,1988-11-23 05:34:00,1,48.83,126.0,73.0,8.05,13.9,14.22,288.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
6,C46206,1988-11-23 06:34:00,1,48.83,126.0,73.0,8.51,17.5,16.00,281.0,...,0.707,13.729,83.1,0.867,0.793,4.366,70.9,0.7644,0.6795,1988-11-23 06:00:00
7,C46206,1988-11-23 07:34:00,1,48.83,126.0,73.0,8.70,13.9,15.06,297.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8,C46206,1988-11-23 08:34:00,1,48.83,126.0,73.0,7.23,13.3,16.00,283.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
9,C46206,1988-11-23 09:34:00,1,48.83,126.0,73.0,7.32,12.2,16.00,289.0,...,0.800,14.450,82.6,1.161,1.093,4.952,67.1,0.7056,0.6109,1988-11-23 09:00:00


In [23]:
main_df['Datetime_x'].max()

Timestamp('2016-01-01 20:38:00')

In [24]:
main_df.shape

(199026, 38)

In [25]:
main_df.tail(10)

,STN_ID,Datetime_x,Q_FLAG,LATITUDE,LONGITUDE,DEPTH,VWH$,VCMX,VTP$,WDIR,...,ETTSw,TPSw,VMDSw,MO1,MO2,HS,DMDIR,ANGSPR,INLINE,Datetime_y
199016,C46206,2016-01-01 11:38:00,1,48.83,126.0,72.0,2.8,4.2,14.2,124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
199017,C46206,2016-01-01 12:38:00,1,48.83,126.0,72.0,2.9,4.7,6.4,121.0,...,0.255,13.984,73.8,0.330,0.382,2.426,87.8,0.3785,0.7548,2016-01-01 12:00:00
199018,C46206,2016-01-01 13:38:00,1,48.83,126.0,72.0,3.2,5.5,6.7,116.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
199019,C46206,2016-01-01 14:38:00,1,48.83,126.0,72.0,3.0,6.2,6.6,114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
199020,C46206,2016-01-01 15:38:00,1,48.83,126.0,72.0,3.0,5.7,6.6,121.0,...,0.214,14.541,73.7,0.374,0.446,2.503,90.4,0.2799,0.7389,2016-01-01 15:00:00
199021,C46206,2016-01-01 16:38:00,1,48.83,126.0,72.0,2.7,4.6,15.1,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
199022,C46206,2016-01-01 17:38:00,1,48.83,126.0,72.0,2.9,4.2,15.1,125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
199023,C46206,2016-01-01 18:38:00,1,48.83,126.0,72.0,2.8,5.1,6.9,133.0,...,0.185,14.839,73.1,0.411,0.496,2.585,275.6,0.3261,0.7159,2016-01-01 18:00:00
199024,C46206,2016-01-01 19:38:00,1,48.83,126.0,72.0,2.8,4.8,6.6,129.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
199025,C46206,2016-01-01 20:38:00,1,48.83,126.0,72.0,2.8,3.9,14.2,123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [26]:
#use forward fill to impute NaN values with same value for 3 hour interval. 
main_df = main_df.fillna(method='ffill')

In [29]:
main_df.head(20)

,STN_ID,Datetime_x,Q_FLAG,LATITUDE,LONGITUDE,DEPTH,VWH$,VCMX,VTP$,WDIR,...,ETTSw,TPSw,VMDSw,MO1,MO2,HS,DMDIR,ANGSPR,INLINE,Datetime_y
0,C46206,1988-11-22 22:34:00,1,48.83,126.0,73.0,5.42,8.4,11.64,234.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,C46206,1988-11-23 00:33:00,1,48.83,126.0,73.0,5.89,9.0,14.22,263.0,...,0.844,12.229,63.7,0.682,0.564,3.981,67.0,0.8075,0.7153,1988-11-23 00:00:00
2,C46206,1988-11-23 01:34:00,1,48.83,126.0,73.0,6.49,10.5,12.80,277.0,...,0.844,12.229,63.7,0.682,0.564,3.981,67.0,0.8075,0.7153,1988-11-23 00:00:00
3,C46206,1988-11-23 02:34:00,1,48.83,126.0,73.0,7.34,12.1,15.06,275.0,...,0.844,12.229,63.7,0.682,0.564,3.981,67.0,0.8075,0.7153,1988-11-23 00:00:00
4,C46206,1988-11-23 04:34:00,1,48.83,126.0,73.0,7.71,13.0,14.22,295.0,...,0.844,12.229,63.7,0.682,0.564,3.981,67.0,0.8075,0.7153,1988-11-23 00:00:00
5,C46206,1988-11-23 05:34:00,1,48.83,126.0,73.0,8.05,13.9,14.22,288.0,...,0.844,12.229,63.7,0.682,0.564,3.981,67.0,0.8075,0.7153,1988-11-23 00:00:00
6,C46206,1988-11-23 06:34:00,1,48.83,126.0,73.0,8.51,17.5,16.00,281.0,...,0.707,13.729,83.1,0.867,0.793,4.366,70.9,0.7644,0.6795,1988-11-23 06:00:00
7,C46206,1988-11-23 07:34:00,1,48.83,126.0,73.0,8.70,13.9,15.06,297.0,...,0.707,13.729,83.1,0.867,0.793,4.366,70.9,0.7644,0.6795,1988-11-23 06:00:00
8,C46206,1988-11-23 08:34:00,1,48.83,126.0,73.0,7.23,13.3,16.00,283.0,...,0.707,13.729,83.1,0.867,0.793,4.366,70.9,0.7644,0.6795,1988-11-23 06:00:00
9,C46206,1988-11-23 09:34:00,1,48.83,126.0,73.0,7.32,12.2,16.00,289.0,...,0.800,14.450,82.6,1.161,1.093,4.952,67.1,0.7056,0.6109,1988-11-23 09:00:00


In [31]:
main_df.isnull().sum()

STN_ID        0
Datetime_x    0
Q_FLAG        0
LATITUDE      0
LONGITUDE     0
DEPTH         0
VWH$          0
VCMX          0
VTP$          0
WDIR          0
WSPD          0
GSPD          0
WDIR.1        0
WSPD.1        0
GSPD.1        0
ATMS          0
DRYT          0
SSTP          0
YEAR          0
DATE          0
WD            1
WS            1
ETOT          1
TP            1
VMD           1
ETTSea        1
TPSea         1
VMDSea        1
ETTSw         1
TPSw          1
VMDSw         1
MO1           1
MO2           1
HS            1
DMDIR         1
ANGSPR        1
INLINE        1
Datetime_y    1
dtype: int64

In [32]:
#use back fill to impute Null values in the first row from the row below
main_df = main_df.fillna(method='bfill')


In [33]:
main_df.isnull().sum()

STN_ID        0
Datetime_x    0
Q_FLAG        0
LATITUDE      0
LONGITUDE     0
DEPTH         0
VWH$          0
VCMX          0
VTP$          0
WDIR          0
WSPD          0
GSPD          0
WDIR.1        0
WSPD.1        0
GSPD.1        0
ATMS          0
DRYT          0
SSTP          0
YEAR          0
DATE          0
WD            0
WS            0
ETOT          0
TP            0
VMD           0
ETTSea        0
TPSea         0
VMDSea        0
ETTSw         0
TPSw          0
VMDSw         0
MO1           0
MO2           0
HS            0
DMDIR         0
ANGSPR        0
INLINE        0
Datetime_y    0
dtype: int64

In [34]:
main_df.to_csv('combined_buoy_hindcast.csv', index=False)
